In [ ]:
wb = 24


In [ ]:
from math import cos, sin
def smallDriveStep(leftX, leftY, rightX, rightY, heading, leftDist, rightDist):
    # returns the new values for left wheel position, right wheel position, and heading
    # only works if leftDist and rightDist are both small relative to the wb
    newLeftX = cos(heading)*leftDist + leftX
    newLeftY = sin(heading)*leftDist + leftY
    newRightX = cos(heading)*rightDist + rightX
    newRightY = sin(heading)*rightDist + rightY
    newHeading = (-leftDist/wb + rightDist/wb) + heading
    return newLeftX, newLeftY, newRightX, newRightY, newHeading

In [ ]:
smallDriveStep(0,wb/2,0,-wb/2,0, 4, 4)

In [ ]:
from math import pi
def driveStep(leftX, leftY, rightX, rightY, heading, leftDist, rightDist):
    # define "small" to be .01  * wb
    if leftDist< .001*wb and rightDist < .001*wb:
        return smallDriveStep(leftX, leftY, rightX, rightY, heading, leftDist, rightDist)
    nSteps = int(max(leftDist, rightDist)/(.001*wb) + 1)
    for step in range(nSteps):
        leftX, leftY, rightX, rightY, heading = smallDriveStep(leftX,
            leftY, rightX, rightY, heading, leftDist/nSteps, rightDist/nSteps)
    return leftX, leftY, rightX, rightY, heading

In [ ]:
driveStep(0,wb/2,0,-wb/2,0, 4, 4)

In [ ]:
driveStep(0,wb/2,0,-wb/2,0, 0, wb*pi/2)

In [ ]:
pi/2

In [ ]:
class Robot(object):
    def __init__(self, wheelbase, maxSpeed):
        self.maxSpeed = maxSpeed
        self.wheelbase = wheelbase
        self.position = (0, 0, 0) # heading, X, Y
        self.throttles = (0,0) # left, Right
        self.counters = (0, 0) # Left, Right
        self.enabled = False
    def __repr__(self):
        heading, x, y = self.position
        leftT, rightT = self.throttles
        leftC, rightC = self.counters
        return f'Robot: \n  Heading, x, y: {heading}, {x}, {y}, \n  Throttles: {leftT}, {rightT}\n  Counters: {leftC}, {rightC}'
    def setPosition(self, position):
        self.position = position
    def getPosition(self): return self.position
    def getThrottles(self): return self.throttles
    def setThrottles(self, throttles): 
        self.throttles = throttles
    def getWheelPositions(self):
        # returns leftX, leftY, rightX, rightY
        (heading, x, y) = self.position
        axleHeading = heading + pi/2
        leftX = cos(axleHeading)*self.wheelbase/2 + x
        leftY = sin(axleHeading)*self.wheelbase/2 + y
        rightX = -cos(axleHeading)*self.wheelbase/2 + x
        rightY = -sin(axleHeading)*self.wheelbase/2 + y
        return (leftX, leftY, rightX, rightY)
    def updatePosition(self, deltaT):
        (heading, x, y) = self.position
        leftX, leftY, rightX, rightY = self.getWheelPositions()
        leftDist = self.leftThrottle*self.maxspeed*deltaT
        rightDist = self.rightThrottle*self.maxspeed*deltaT
        leftX, leftY, rightX, rightY, heading = driveStep(leftX, leftY, rightX, rightY, heading, leftDist, rightDist)
        # store the new position based on the wheel positions
        self.setPosition((heading, (leftX+rightX)/2, (leftY+rightY)/2))

In [ ]:
r = Robot(24, 100)
r

In [ ]:
class AutoProg:
    # stages
    def __init__(self, robot):
        self.robot = robot
        self.done = True
    # step returns True if the program is finished, False if not
    def step(self, deltaT):
        if self.done: 
            self.done = False
            self.init()
        if not self.done: 
            self.runStep(deltaT)
        return self.done
    def init(self): # override me
        pass
    def runStep(self,deltaT): # override me; set self.done when finished
        self.done = True
        
    
class ConditionalProg(AutoProg):
    # test is a function returning an integer in range(len(progs))
    def __init__(self, robot, test, progs):
        super().__init__(robot)
        self.test = test
        self.progs = progs
    def init(self):
        self.activeProg = self.progs[self.test()]
    def runStep(self, deltaT):
        self.done = self.activeProg.step(deltaT)
    
class SequentialProg(AutoProg):
    def __init__(self, robot, progs):
        super().__init__(robot)
        self.progs = progs
    def init(self):
        self.pos = 0
    def runStep(self, deltaT):
        if self.pos >= len(self.progs):
            self.done = True
            return
        if self.progs[self.pos].step(deltaT):
            self.pos += 1
        
class FirstToFinish(AutoProg):
    def __init__(self, robot, progs):
        super().__init__(robot)
        self.progs = progs
    def init(self):
        pass # nothing to do
    def runStep(self, deltaT):
        for p in self.progs:
            if p.step(deltaT):
                self.done = True
            break

class ParallelProg(AutoProg):
    def __init__(self, robot, progs):
        super().__init__(robot)
        self.progs = progs
    def init(self):
        self.finishedCount = 0
        for p in self.progs:
            p.done = False
            p.init()
            if p.done: self.finishedCount += 1
    def runStep(self, deltaT):
        for p in self.progs:
            if not p.done: 
                done = p.step(deltaT)
                if done: self.finishedCount += 1
        if self.finishedCount == len(self.progs):
            self.done = True

from time import clock   
from IPython.core.debugger import set_trace
class Timeout(AutoProg):
    def __init__(self, robot, timeout):
        super().__init__(robot)
        self.timeout = timeout
    def init(self):
        self.finishTime = clock()+self.timeout
    def runStep(self, deltaT):
        if clock() >= self.finishTime:
            self.done = True

In [ ]:
from time import sleep
# test programs with a 10 second limit
def progTest(p, title):
    print(f'{title} Start time {clock()}')
    for i in range(1000):
        if p.step(.01):
            break
        sleep(0.01)
    print(f'{title} Finish time {clock()}')

In [ ]:
r = Robot(24, 100)
twoSeconds = Timeout(r, 2)
fourSeconds = Timeout(r, 4)
sequential = SequentialProg(r, [twoSeconds, fourSeconds])
parallel = ParallelProg(r, [twoSeconds, fourSeconds])
firstFinish = FirstToFinish(r, [twoSeconds, fourSeconds])
progTest(twoSeconds, "Two Seconds")
progTest(fourSeconds, "Four Seconds")
progTest(sequential, "Sequential")
progTest(parallel, "Parallel")
progTest(firstFinish, "FirstToFinish")
nullProg = AutoProg(r)
progTest(nullProg, 'Null Program')

In [ ]:
progTest(firstFinish, "first")

In [ ]:
from collections import namedtuple
Position = namedtuple('Position', 'heading x y')

In [ ]:
Position(0,0,0)

In [ ]:
_

In [ ]:
h,x,y